In [37]:
from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI
import os
from langchain.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer # type: ignore
load_dotenv()

True

In [2]:
llm = AzureChatOpenAI(openai_api_version=os.environ.get("AZURE_OPENAI_VERSION", "2023-07-01-preview"),
    azure_deployment=os.environ.get("AZURE_OPENAI_DEPLOYMENT", "gpt4chat"),
    azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT", "https://gpt-4-trails.openai.azure.com/"),
    api_key=os.environ.get("AZURE_OPENAI_KEY"))

In [4]:
file_path = "biomed.pdf"
loader = PyPDFLoader(file_path)
pages = loader.load()

In [5]:
len(pages)

12

In [13]:
print(pages[0])

page_content='Citation: Pullakhandam, S.; McRoy, S.\nClassification and Explanation of Iron\nDeficiency Anemia from Complete\nBlood Count Data Using Machine\nLearning. BioMedInformatics 2024 ,4,\n661–673. https://doi.org/10.3390/\nbiomedinformatics4010036\nAcademic Editors: Pentti Nieminen\nand Jörn Lötsch\nReceived: 12 January 2024\nRevised: 15 February 2024\nAccepted: 22 February 2024\nPublished: 1 March 2024\nCopyright: ©2024 by the authors.\nLicensee MDPI, Basel, Switzerland.\nThis article is an open access article\ndistributed under the terms and\nconditions of the Creative Commons\nAttribution (CC BY) license (https://\ncreativecommons.org/licenses/by/\n4.0/).\nArticle\nClassification and Explanation of Iron Deficiency Anemia from\nComplete Blood Count Data Using Machine Learning\nSiddartha Pullakhandam and Susan McRoy *\nDepartment of Computer Science, University of Wisconsin-Milwaukee, Milwaukee, WI 53211, USA;\npullakh2@uwm.edu\n*Correspondence: mcroy@uwm.edu; Tel.: +1-414-229

In [28]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=250,
    chunk_overlap=50,
    length_function=len,
)

In [29]:
pages_text = [pages.page_content for pages in pages]
# the type of pages is documents but we need to convert them into list of strings for splitting the text.

In [33]:
documents = splitter.create_documents(pages_text)
# splitting the text into chunks of 250 characters and 50 characters overlap.

In [34]:
type(documents[0])

langchain_core.documents.base.Document

In [35]:
len(documents)

277

In [ ]:
"""
You are a language model designed to evaluate the responses of this documentation query system.
You will use a rating scale of 0 to 10, 0 being poorest response and 10 being the best.
Responses with “not specified” or “no specific mention” or “rephrase question” or “unclear” or no documents returned or empty response are considered poor responses.
Responses where the question appears to be answered are considered good.
Responses that contain detailed answers are considered the best.
Also, use your own judgement in analyzing if the question asked is actually answered in the response. Remember that a response that contains a request to “rephrase the question” is usually a non-response.
Please rate the question/response pair entered. Only respond with the rating. No explanation necessary. Only integers.
"""